# Imports

In [1]:
import pandas as pd
import numpy as np
import json



# Get Data

In [68]:
batch_amount = 3
for i in range(batch_amount):
    #json_file_paths.append(f'../raw_data/playlist_data/mpd.slice.{i*1000}-{999+ i * 1000}.json')
    print((f'../raw_data/playlist_data/mpd.slice.{i*1000}-{999+ i * 1000}.json'))

../raw_data/playlist_data/mpd.slice.0-999.json
../raw_data/playlist_data/mpd.slice.1000-1999.json
../raw_data/playlist_data/mpd.slice.2000-2999.json


In [69]:
# Specify the path to your JSON file
json_file_paths = []
for i in range(3):
    json_file_paths.append(f'../raw_data/playlist_data/mpd.slice.{i*1000}-{999+ i * 1000}.json')

# Read the JSON file
json_data = []
for i in range(len(json_file_paths)):
    with open(json_file_paths[i], 'r') as file:
        json_data.append(json.load(file))

In [71]:
json_data[0]['playlists'][0]['tracks'][0]['artist_name']


'Missy Elliott'

In [77]:
track_sum_per_batch = 0
batch = 0
for i in range(len(json_data[batch]['playlists'])):
    playlist_i = json_data[batch]['playlists'][i]['tracks']
    track_sum_per_batch += len(playlist_i)
track_sum_per_batch

65749

In [76]:
track_sum = 0
for i in range(len(json_data)):
    json_i = json_data[i]['playlists']
    for j in range(len(json_i)):
        playlist = json_data[i]['playlists'][j]['tracks']
        track_sum += len(playlist)
track_sum

199874

In [79]:
tracklist_per_batch = []
batch = 0

for i in range(len(json_data[batch]['playlists'])):
    playlist_i = json_data[batch]['playlists'][i]['tracks']
    for j in range(len(playlist_i)):
        tracklist_per_batch.append(f'{i}' + ': ' + playlist_i[j]['artist_name'] + ' - ' + playlist_i[j]['track_name'])
        
tracklist_per_batch

['0: Missy Elliott - Lose Control (feat. Ciara & Fat Man Scoop)',
 '0: Britney Spears - Toxic',
 '0: Beyoncé - Crazy In Love',
 '0: Justin Timberlake - Rock Your Body',
 "0: Shaggy - It Wasn't Me",
 '0: Usher - Yeah!',
 '0: Usher - My Boo',
 '0: The Pussycat Dolls - Buttons',
 "0: Destiny's Child - Say My Name",
 '0: OutKast - Hey Ya! - Radio Mix / Club Mix',
 '0: Nelly Furtado - Promiscuous',
 '0: Jesse McCartney - Right Where You Want Me - Radio Edit Version',
 '0: Jesse McCartney - Beautiful Soul',
 "0: Jesse McCartney - Leavin'",
 '0: Cassie - Me & U',
 '0: Omarion - Ice Box',
 '0: Avril Lavigne - Sk8er Boi',
 '0: Chris Brown - Run It!',
 '0: Beyoncé - Check On It - feat. Bun B and Slim Thug',
 "0: Destiny's Child - Jumpin', Jumpin'",
 '0: Sheryl Crow - Soak Up The Sun',
 '0: The Black Eyed Peas - Where Is The Love?',
 "0: Bowling For Soup - Stacy's Mom",
 '0: The Click Five - Just The Girl',
 '0: Chris Brown - Yo (Excuse Me Miss)',
 '0: Jonas Brothers - Year 3000',
 '0: Lil Mama -

In [83]:
tracklist = []
for k in range (len(json_data)):
    for i in range(len(json_data[k]['playlists'])):
        playlist_i = json_data[k]['playlists'][i]['tracks']
        for j in range(len(playlist_i)):
            tracklist.append(f'{k}' + '/' + f'{i}' + ': ' + playlist_i[j]['artist_name'] + ' - ' + playlist_i[j]['track_name'])
tracklist

['0/0: Missy Elliott - Lose Control (feat. Ciara & Fat Man Scoop)',
 '0/0: Britney Spears - Toxic',
 '0/0: Beyoncé - Crazy In Love',
 '0/0: Justin Timberlake - Rock Your Body',
 "0/0: Shaggy - It Wasn't Me",
 '0/0: Usher - Yeah!',
 '0/0: Usher - My Boo',
 '0/0: The Pussycat Dolls - Buttons',
 "0/0: Destiny's Child - Say My Name",
 '0/0: OutKast - Hey Ya! - Radio Mix / Club Mix',
 '0/0: Nelly Furtado - Promiscuous',
 '0/0: Jesse McCartney - Right Where You Want Me - Radio Edit Version',
 '0/0: Jesse McCartney - Beautiful Soul',
 "0/0: Jesse McCartney - Leavin'",
 '0/0: Cassie - Me & U',
 '0/0: Omarion - Ice Box',
 '0/0: Avril Lavigne - Sk8er Boi',
 '0/0: Chris Brown - Run It!',
 '0/0: Beyoncé - Check On It - feat. Bun B and Slim Thug',
 "0/0: Destiny's Child - Jumpin', Jumpin'",
 '0/0: Sheryl Crow - Soak Up The Sun',
 '0/0: The Black Eyed Peas - Where Is The Love?',
 "0/0: Bowling For Soup - Stacy's Mom",
 '0/0: The Click Five - Just The Girl',
 '0/0: Chris Brown - Yo (Excuse Me Miss)',

In [81]:
len(tracklist)

199874

In [86]:
tracklist

['0/0: Missy Elliott - Lose Control (feat. Ciara & Fat Man Scoop)',
 '0/0: Britney Spears - Toxic',
 '0/0: Beyoncé - Crazy In Love',
 '0/0: Justin Timberlake - Rock Your Body',
 "0/0: Shaggy - It Wasn't Me",
 '0/0: Usher - Yeah!',
 '0/0: Usher - My Boo',
 '0/0: The Pussycat Dolls - Buttons',
 "0/0: Destiny's Child - Say My Name",
 '0/0: OutKast - Hey Ya! - Radio Mix / Club Mix',
 '0/0: Nelly Furtado - Promiscuous',
 '0/0: Jesse McCartney - Right Where You Want Me - Radio Edit Version',
 '0/0: Jesse McCartney - Beautiful Soul',
 "0/0: Jesse McCartney - Leavin'",
 '0/0: Cassie - Me & U',
 '0/0: Omarion - Ice Box',
 '0/0: Avril Lavigne - Sk8er Boi',
 '0/0: Chris Brown - Run It!',
 '0/0: Beyoncé - Check On It - feat. Bun B and Slim Thug',
 "0/0: Destiny's Child - Jumpin', Jumpin'",
 '0/0: Sheryl Crow - Soak Up The Sun',
 '0/0: The Black Eyed Peas - Where Is The Love?',
 "0/0: Bowling For Soup - Stacy's Mom",
 '0/0: The Click Five - Just The Girl',
 '0/0: Chris Brown - Yo (Excuse Me Miss)',

In [ ]:
for track in tracklist:
    if 